<a href="https://colab.research.google.com/github/7ft10/JiraExporter/blob/main/DataFlows - Server/Jira_Foundation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#! Library Import

import os 
import importlib.util

if importlib.util.find_spec("pandas") is None:	
	os.system("pip install pandas")

if importlib.util.find_spec("dotenv") is None:	
	os.system("pip install --quiet openai python-dotenv")

In [3]:
#! pandas Config

import pandas as pd
import requests
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

pd.options.mode.use_inf_as_na = True
#pd.set_option("display.max_rows", 10)
#pd.set_option("display.expand_frame_repr", True)
#pd.set_option('display.width', 1000)
#pd.options.display.max_seq_items = 200000
#pd.options.display.max_rows = 10

In [4]:
#! Parameters

import os 
import dotenv
import importlib.util

if importlib.util.find_spec("google.colab") is not None: ## if using google colab
    if not os.path.exists('.env'):
        from google.colab import files
        uploaded = files.upload()
        file_name = list(uploaded.keys())[0]
        try:
            os.rename(file_name, '.env')
        except:
            pass

try:
    load_dotenv('.env')

    Host = os.getenv('SECRETS_HOST') 
    Username = os.getenv('SECRETS_USERNAME') 
    Password = os.getenv('SECRETS_PASSWORD') 
except: 
    pass 

if Host is None or Host == "":
    Host = input("Enter Host")

if Username is None or Username == "":
    Username = input("Enter Username")

if Password is None or Password == "":
    Password = input("Enter Password")

display("Host: " + Host)

ValidProjectCategories = ["'Portfolio'"]
display("ValidProjectCategories: " + ','.join(ValidProjectCategories))

'Host: https://jira.budgetdirect.com.au/'

"ValidProjectCategories: 'Portfolio'"

In [5]:
#! Functions

import time
import base64
import pandas as pd
import re 
import requests
import warnings
from functools import reduce
    
def _ExpandColumn(self:pd.DataFrame, colName:str, columnsToExpand = [], prefix:str = "Prefix", sentenceCase:bool = True) -> pd.DataFrame:
    if (prefix == "Prefix"):
        prefix = colName + " "
        with warnings.catch_warnings():
          warnings.simplefilter(action='ignore', category=FutureWarning)
          expandedCols = self[colName].apply(lambda x: pd.Series(x).add_prefix(prefix))
        columnsToExpand = [prefix + c for c in columnsToExpand]
    else:
        expandedCols = self[colName].apply(lambda x: pd.Series(x))
    
    if len(columnsToExpand) > 0:        
        expandedCols = expandedCols[columnsToExpand]
    
    if sentenceCase:
        expandedCols.columns = [fnSentenceCase(c) for c in expandedCols.columns] 

    return pd.concat([self.drop(colName, axis=1), expandedCols], axis=1)

pd.DataFrame.expand = _ExpandColumn

def fnSentenceCase(s):
    s = (' '.join(dict.fromkeys(s.split())))  # remove duplicate words
    s = s.replace("0", "") # remove "0" 
    s = s.strip()
    return ' '.join([x.capitalize() for x in re.sub(r"([A-Z])", r" \1", s).split()]) # sentence case

def _SentenceCaseColumns(self:pd.DataFrame) -> pd.DataFrame: 
    self.columns = [fnSentenceCase(c) for c in self.columns] 
    return self

pd.DataFrame.sentence_case_columns = _SentenceCaseColumns

def fnGetDefaultHeaders():
    return {
        "content-type": "application/json",
        "authorization": "Basic " + base64.b64encode((Username + ":" + Password).encode()).decode(),
        "retry-after": "120"
    }

def fnSearch(jql, fields = None, expand = None):
    def ApiCall(startAt) :
        url = "/rest/api/latest/search"
        headers = fnGetDefaultHeaders()
        defaultContents = {
            "startAt": startAt,
            "maxResults": "2",
            "jql": jql
        }
        if fields is not None:
            defaultContents["fields"] = fields.tolist()
        if expand is not None and expand != "":
            defaultContents["expand"] = expand        
        response = requests.post(Host + url, headers = headers, json = defaultContents, verify=False)
        return response.json()
    values = fnAPI(ApiCall)
    if len(values.index) > 1:
        return values
    else:
        None

def fnGetIssueTypeFields(IssueTypes) -> pd.DataFrame:
    def ApiCall(startAt) :
        url = "rest/api/latest/issue/createmeta"
        headers = fnGetDefaultHeaders()
        params = {
            "expand": "projects.issuetypes.fields",
            "projectKeys": ','.join(fnGetValidProjectKeys()["key"].values),
            "issuetypeNames": ','.join(IssueTypes).replace("'", "")
        }
        response = requests.get(Host + url, headers = headers, params = params, verify=False)
        return response.json()
    
    df = fnAPI(ApiCall)
    df = df.drop(["expand"], axis=1)
    df = df.explode("projects")
    df = df.expand("projects", [], None, False)    
    try:
        df = df[["issuetypes"]]
    except: 
        raise Exception("No issue metadata - check the valid project categories are correct")
    df = df.explode("issuetypes")
    df = df.expand("issuetypes", [], None, False)
    df = df[["fields"]]
    df = df.expand("fields", [], None, False)
    df = df.loc[:,~df.columns.duplicated()]

    values = []
    for x in df.columns:        
        try:            
            valid:pd.DataFrame = pd.DataFrame( df[~df[x].isnull()] )[[x]].iloc[0].get(0)
            values.append({
                "fieldId": valid['key'] if "key" in valid else valid["fieldId"],
                "name": valid['name'],
                "schema_type": valid['schema']['type'],
                "required": valid['required']
            })
        except:
            #display(x)
            pass
    values.append({ "fieldId": 'status', "name": 'Status', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'created', "name": 'Created', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'updated', "name": 'Updated', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'resolution', "name": 'Resolution', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'resolutiondate', "name": 'Resolution Date', "schema_type": 'date', "required": False })
    values.append({ "fieldId": 'lastViewed', "name": 'Last Viewed', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'id', "name": 'Id', "schema_type": 'number', "required": True })
    values.append({ "fieldId": 'key', "name": 'Key', "schema_type": 'string', "required": True })
    df = pd.DataFrame(values)
    df = df.drop_duplicates().sort_values("fieldId")
    return df 

def fnGetValidProjectKeys() -> pd.DataFrame:
    def ApiCall(startAt) :
        url = "/rest/api/latest/project"
        headers = fnGetDefaultHeaders()
        params = { }
        response = requests.get(Host + url, headers = headers, params = params, verify=False)
        return response.json()
    df = fnAPI(ApiCall)
    df = df.expand("projectCategory")
    if 'ValidProjectCategories' in globals() and len(ValidProjectCategories) > 0:
        df = df.loc[df['Project Category Name'].isin(ValidProjectCategories) | ("'" + df['Project Category Name'] + "'").isin(ValidProjectCategories)]
    return df[["key"]]

def fnAPI(webRequestDelegate, startAt = 0) -> pd.DataFrame:
    def flatten_reduce_lambda(frm):
        try:
            return list(reduce(lambda x, y: x + y, frm, []))         
        except:
            return list(reduce(lambda x, y: x + y, [frm], [])) 
    def innerGetResults(webRequestDelegate, startAt = 0):
        results = webRequestDelegate(startAt)
        if isinstance(results, dict) and "total" in results and "maxResults" in results:
            if startAt + results["maxResults"] < results["total"]:
                return [results] + innerGetResults(webRequestDelegate, startAt + results["maxResults"])
            else:
                return [results]
        else:
            return [results]
    Source = flatten_reduce_lambda(innerGetResults(webRequestDelegate, startAt))
    df = pd.DataFrame(Source)
    return df

def fnGetTimeZoneOffset() -> str:
    offset = (time.timezone if (time.localtime().tm_isdst == 0) else time.altzone) / 60 / 60 * -1
    return ("" if(offset) < 0 else "+") + str(int((offset - (offset % 1)))).zfill(2) + ":" + str(int((offset % 1) * 60)).zfill(2)

goldenDF = None
globals()['goldenDF'] = None 

def fnGetGoldenCopy() -> pd.DataFrame:
    if 'goldenDF' not in globals() or goldenDF is None: 
        raise Exception("Base data frame not loaded") 
    else:
        df:pd.DataFrame = goldenDF.copy(deep = True)
        if df is None or len(df.index) == 0:
            raise Exception ("No results")
    return df

def exit():
    class StopExecution(Exception):
        def _render_traceback_(self):
            []
    raise StopExecution


In [6]:
#! Projects

from IPython.display import display
import requests
from pandas.core.dtypes.dtypes import DatetimeTZDtype

def ApiCall(startAt) :
    url = "/rest/api/latest/project"
    headers = fnGetDefaultHeaders()
    params = {
        "startAt": startAt,
        "expand": "projectKeys"
    }
    response = requests.get(Host + url, headers = headers, params = params, verify=False)
    return response.json()

df = fnAPI(ApiCall)  
df = df.drop(["avatarUrls", "expand", "self", "projectKeys"], axis=1).sentence_case_columns()
df = df.expand("Project Category", ["id", "name"])
df["Is Valid"] = (
    df['Project Category Name'].isin(ValidProjectCategories) | ("'" + df['Project Category Name'] + "'").isin(ValidProjectCategories) if 'ValidProjectCategories' in globals() and len(ValidProjectCategories) > 0 else True
)

df = df.sentence_case_columns().convert_dtypes().infer_objects()
df["Id"] = df["Id"].astype('Int64')
df["Project Category Id"] = df["Project Category Id"].astype('Int64')

display(df.dtypes)
display(df.head())

Id                                Int64
Key                      string[python]
Name                     string[python]
Project Type Key         string[python]
Project Category Id               Int64
Project Category Name    string[python]
Is Valid                        boolean
dtype: object

,Id,Key,Name,Project Type Key,Project Category Id,Project Category Name,Is Valid
0,21420,AGP,A&G Group Portfolio,software,10930,Portfolio,True
1,21228,AGQL,AGGI Spirit,software,<NA>,<NA>,False
2,10213,AGG,Aggregator,software,10130,CtM,False
3,13320,API,API,software,10230,Online Applications,False
4,22220,ARC,Architecture Team,software,10010,Development,False


In [7]:
#! Project Keys

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/project"
    headers = fnGetDefaultHeaders()
    params = {
        "startAt": startAt,
        "expand": "projectKeys"
    }
    response = requests.get(Host + url, headers = headers, params = params, verify=False)
    return response.json()

df = fnAPI(ApiCall).sentence_case_columns()

df = df[["Id", "Key", "Project Keys"]]
df = df.explode("Project Keys")

df = df.sentence_case_columns().convert_dtypes().infer_objects()
df["Id"] = df["Id"].astype('Int64')

display(df.dtypes)
display(df.head())

Id                       Int64
Key             string[python]
Project Keys    string[python]
dtype: object

,Id,Key,Project Keys
0,21420,AGP,AGP
1,21228,AGQL,AGQL
2,10213,AGG,AGG
3,13320,API,API
4,22220,ARC,ARC


In [8]:
#! Project Categories

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/projectCategory"
    headers = fnGetDefaultHeaders()
    params = {
        "startAt": startAt
    }
    response = requests.get(Host + url, headers = headers, params = params, verify=False)
    return response.json()

df = fnAPI(ApiCall)  
df = df.drop(["self"], axis=1)

df = df.sentence_case_columns().convert_dtypes().infer_objects()
df["Id"] = df["Id"].astype('Int64')

list_row = [0, "No Category", "No Category"]
df.loc[len(df)] = list_row
df = df.sort_values('Id')

display(df.dtypes)
display(df.head())

Id              Int64
Description    object
Name           object
dtype: object

,Id,Description,Name
27,0,No Category,No Category
7,10010,Development tasks (not associated with a project),Development
11,10011,Development projects which have been finished,Finished
13,10012,IT Infrastructure Projects,Infrastructure
3,10013,Development tasks (associated with a project),Complex Projects


In [9]:
#! Issue Types

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/issuetype"
    headers = fnGetDefaultHeaders()
    params = {
        "startAt": startAt
    }
    response = requests.get(Host + url, headers = headers, params = params, verify=False)
    return response.json()

df = fnAPI(ApiCall)  
df = df.drop(["self"], axis=1)
df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
df["Id"] = df["Id"].astype('Int64')

df = df.sort_values('Name').reset_index(drop=True)

display(df.dtypes)
display(df.head())

Id                      Int64
Description    string[python]
Icon Url       string[python]
Name           string[python]
Subtask               boolean
Avatar Id               Int64
dtype: object

,Id,Description,Icon Url,Name,Subtask,Avatar Id
0,25,A request for new or existing 1300/1800 number...,https://jira.budgetdirect.com.au/secure/viewav...,1300/1800 Number Administration,False,13638
1,13608,"Setup of Livechat, etc.",https://jira.budgetdirect.com.au/secure/viewav...,3rd party integration,False,13620
2,13208,,https://jira.budgetdirect.com.au/secure/viewav...,A/B Test,False,13624
3,10204,,https://jira.budgetdirect.com.au/images/icons/...,AM Run Log Checks,False,<NA>
4,16311,,https://jira.budgetdirect.com.au/secure/viewav...,AV,False,13620


In [10]:
#! Project Types

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/project/type"
    headers = fnGetDefaultHeaders()
    params = {}
    response = requests.get(Host + url, headers = headers, params = params, verify=False)
    return response.json()

df = fnAPI(ApiCall)  
df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
df = df.sort_values('Key').reset_index(drop=True)

display(df.dtypes)
display(df.head())

Key                     string[python]
Formatted Key           string[python]
Description I18n Key    string[python]
Icon                    string[python]
Color                   string[python]
dtype: object

,Key,Formatted Key,Description I18n Key,Icon,Color
0,business,Business,jira.project.type.business.description,PHN2ZyB2ZXJzaW9uPSIxIiB4bWxucz0iaHR0cDovL3d3dy...,#0065FF
1,software,Software,jira.project.type.software.description,PHN2ZyB2ZXJzaW9uPSIxIiBpZD0iV2Fyc3R3YV8xIiB4bW...,#FF991F


In [11]:
#! Resolutions

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/resolution"
    headers = fnGetDefaultHeaders()
    params = {}
    response = requests.get(Host + url, headers = headers, params = params, verify=False)
    return response.json()

df = fnAPI(ApiCall)  
df = df.drop(["self"], axis=1)
df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
df["Id"] = df["Id"].astype('Int64')
df = df.sort_values('Name').reset_index(drop=True)

display(df.dtypes)
display(df.head())

Id                      Int64
Description    string[python]
Name           string[python]
dtype: object

,Id,Description,Name
0,10202,Question has been answered.,Answered
1,10102,Request has been approved.,Approved
2,5,"All attempts at reproducing this issue failed,...",Cannot Reproduce
3,10203,Question could not be answered.,Could Not Answer
4,10000,Work has been completed on this issue.,Done


In [16]:
#! Status

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/status"
    headers = fnGetDefaultHeaders()
    params = {}
    response = requests.get(Host + url, headers = headers, params = params, verify=False)
    return response.json()

df = fnAPI(ApiCall)  
df = df.drop(["self"], axis=1)
df = df.expand("statusCategory", ["id", "name"])

df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
df["Id"] = df["Id"].astype('Int64')
df = df.sort_values('Name').reset_index(drop=True)

display(df.dtypes)
display(df.head())

Description             string[python]
Icon Url                string[python]
Name                    string[python]
Id                               Int64
Status Category Id               Int64
Status Category Name    string[python]
dtype: object

,Description,Icon Url,Name,Id,Status Category Id,Status Category Name
0,This status is managed internally by Jira Soft...,https://jira.budgetdirect.com.au/,Actions,15372,4,In Progress
1,,https://jira.budgetdirect.com.au/images/icons/...,Additional Work,10054,4,In Progress
2,,https://jira.budgetdirect.com.au/images/icons/...,Approval Required,10038,2,To Do
3,,https://jira.budgetdirect.com.au/images/icons/...,Approved - Ready for Prioritisation,13066,4,In Progress
4,,https://jira.budgetdirect.com.au/images/icons/...,Assigned,11963,4,In Progress


In [ ]:
# Status Categories

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/statuscategory"
    headers = fnGetDefaultHeaders()
    params = {}
    response = requests.get(Host + url, headers = headers, params = params, verify=False)
    return response.json()

df = fnAPI(ApiCall)  
df = df.drop(["self"], axis=1)

df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
df["Id"] = df["Id"].astype('Int64')
df = df.sort_values('Id').reset_index(drop=True)

display(df.dtypes)
display(df.head())

Id                     Int64
Key           string[python]
Color Name    string[python]
Name          string[python]
dtype: object

,Id,Key,Color Name,Name
0,1,undefined,medium-gray,No Category
1,2,new,blue-gray,To Do
2,3,done,green,Done
3,4,indeterminate,yellow,In Progress


In [ ]:
#! Users

from IPython.display import display
import requests
import pandas as pd

def ApiCall(startAt) :
    url = "/rest/api/latest/users/search"
    headers = fnGetDefaultHeaders()
    params = {
        "startAt": startAt,
        "maxResults": "1000"
    }
    response = requests.get(Host + url, headers = headers, params = params, verify=False)
    return response.json()

df = pd.concat([fnAPI(ApiCall, x * 1000) for x in range(0, 10)])
df = df.drop(["self"], axis=1)

df = df.expand("avatarUrls", ["48x48"]).rename(columns={"Avatar Urls 48x48": "Avatar Url"})

df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
df = df.sort_values('Display Name').reset_index(drop=True)

display(df.dtypes)
display(df.head())

Account Id       string[python]
Account Type     string[python]
Email Address    string[python]
Display Name     string[python]
Active                  boolean
Time Zone        string[python]
Locale           string[python]
Avatar Url       string[python]
dtype: object

,Account Id,Account Type,Email Address,Display Name,Active,Time Zone,Locale,Avatar Url
0,qm:0956ac3a-a022-4a10-a3c5-22e5fd6f75cf:5c80c3...,customer,<NA>,2014490606@qq.com,True,<NA>,<NA>,https://avatar-management--avatars.us-west-2.p...
1,6139d99124ba8b00701a07b1,atlassian,aws-support@strategenics.com.au,AWS-SUPPORT@strategenics.com.au,True,<NA>,<NA>,https://secure.gravatar.com/avatar/6fd0cc8a51c...
2,632113a7cd4a82cf977cafdb,atlassian,<NA>,Adrian Hill,True,<NA>,en_GB,https://secure.gravatar.com/avatar/939dc6b1cfe...
3,557057:fa71e034-c716-4a38-96eb-e3d8f5e106ca,atlassian,<NA>,Adven,False,<NA>,en_US,https://avatar-management--avatars.us-west-2.p...
4,5e5c7ada5a495e0c91a9a2a1,atlassian,<NA>,Agnieszka Klyz,True,<NA>,en_GB,https://secure.gravatar.com/avatar/a12986dd346...
...,...,...,...,...,...,...,...,...
1110,qm:0956ac3a-a022-4a10-a3c5-22e5fd6f75cf:3e9f95...,customer,<NA>,wmalovnek@interactit.com.au,True,<NA>,<NA>,https://avatar-management--avatars.us-west-2.p...
1111,qm:0956ac3a-a022-4a10-a3c5-22e5fd6f75cf:5c32c0...,customer,<NA>,yamiko.kambalametore@intertek.com,True,<NA>,<NA>,https://avatar-management--avatars.us-west-2.p...
1112,qm:0956ac3a-a022-4a10-a3c5-22e5fd6f75cf:8c9ec0...,customer,<NA>,yasmineazzaoui2@gmail.com,True,<NA>,<NA>,https://avatar-management--avatars.us-west-2.p...
1113,qm:0956ac3a-a022-4a10-a3c5-22e5fd6f75cf:101b82...,customer,<NA>,yourmail@gmail.com,True,<NA>,<NA>,https://avatar-management--avatars.us-west-2.p...


In [ ]:
#! Priorities

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/priority"
    headers = fnGetDefaultHeaders()
    params = {}
    response = requests.get(Host + url, headers = headers, params = params, verify=False)
    return response.json()

df = fnAPI(ApiCall)  
df = df.drop(["self"], axis=1)

df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
df["Id"] = df["Id"].astype('Int64')
df = df.sort_values('Id').reset_index(drop=True)

display(df.dtypes)
display(df.head())

Status Color    string[python]
Description     string[python]
Icon Url        string[python]
Name            string[python]
Id                       Int64
dtype: object

,Status Color,Description,Icon Url,Name,Id
0,#cc0000,to do,https://strategenics.atlassian.net/images/icon...,Critical,1
1,#ff0000,to do,https://strategenics.atlassian.net/images/icon...,High,2
2,#009900,to do,https://strategenics.atlassian.net/images/icon...,Medium,3
3,#006600,to do,https://strategenics.atlassian.net/images/icon...,Low,4
4,#003300,to do,https://strategenics.atlassian.net/images/icon...,Enhancement,5
